In [6]:
import pandas as pd 
import numpy as np 

In [7]:
file = pd.read_excel(r'groupe5_stationlibre.xlsx' )
file

,station,serie,point_visé,cg,rcg,cd,rcd,moyenne,distancecg,distancecd,distancemoy,Avcg,Avcd,zocd,Av
0,NaN,NaN,topo12,0.0000,NaN,200.0000,NaN,NaN,116.6046,116.5988,NaN,100.5440,299.4784,NaN,NaN
1,M1,1.0,topo3,151.4514,NaN,351.4588,NaN,NaN,93.7784,93.7650,NaN,99.1892,300.8976,NaN,NaN
2,NaN,NaN,topo12,0.0020,NaN,200.0062,NaN,NaN,116.6098,116.6076,NaN,100.6144,299.4772,NaN,NaN
3,NaN,NaN,topo12,50.0000,NaN,250.0000,NaN,NaN,116.6078,116.6078,NaN,100.6104,299.4772,NaN,NaN
4,M1,2.0,topo3,201.4480,NaN,1.4454,NaN,NaN,93.7824,93.7706,NaN,99.1926,300.8940,NaN,NaN
5,NaN,NaN,topo12,50.0010,NaN,250.0016,NaN,NaN,116.6404,116.6032,NaN,100.6164,299.4696,NaN,NaN


In [8]:
import numpy as np  
def rad2grad(angle) : 
    return angle *  200 / (np.pi)
def grad2rad(angle): 
    return angle * (np.pi/200) 
def cosg(angle) : 
    return np.cos(grad2rad(angle))
def sing(angle): 
    return np.sin(grad2rad(angle))
    
def roundd(angle) : 
    if angle > 400 or angle < 0 : 
        angle = angle % 400  
    return round(angle , 4)  
def gisement(x1,y1,x2,y2) -> float  : 
    try : 
        dx = x2 - x1
        dy = y2 - y1 
        if dy == 0 : 
            if dx > 0 : 
                gisement = 100 
            elif dx > 0 : 
                gisement = 300 
        G = rad2grad(np.arctan(dx / dy ))
        if dx == 0  : 
            if dy > 0 : 
                gisement = 0 
            if dy < 0 : 
                gisement = 200 
        else : 
            R = np.abs(dx) /dx 
            S = 1 + np.abs(dy) / dy 
            gisement = G + 200 - 100 * R * S 
        return roundd(gisement) 
    except ZeroDivisionError : 
        return "invalid_input"    

In [9]:
angles = np.zeros(int(len(file.rcg) / 3 ))
rcgg = np.zeros(len(file.rcg))
rcdd = np.zeros(len(file.rcg))
cd = file.cd
cg = file.cg
moyenne = np.zeros(len(file.rcg)) 
n = len(file.rcg) 
for i in range(1 , n  , 3 ) : 
    p = file.cg[i+1] - file.cg[i-1]
    if p> 399  :
        rcgg[i] = roundd(file.cg[i] - (file.cg[i+1]+400+file.cg[i-1])/2)
    else: 
        rcgg[i] =  roundd(file.cg[i] - (file.cg[i+1] +file.cg[i-1])/2)
for i in range(1 , n  , 3 ) : 
    p = file.cd[i+1] - file.cd[i-1]
    if p> 399  :
        rcdd[i] =  roundd(file.cd[i] - (file.cd[i+1]+400+file.cd[i-1])/2)
    else: 
        rcdd[i] =  roundd(file.cd[i] - (file.cd[i+1] +file.cd[i-1])/2)
    moyenne[i] = ((rcdd[i] + rcgg[i]) / 2 )  
    angles[i//3] = moyenne[i]
file.rcg = rcgg 
file.rcd = rcdd 
file.moyenne = moyenne
file

,station,serie,point_visé,cg,rcg,cd,rcd,moyenne,distancecg,distancecd,distancemoy,Avcg,Avcd,zocd,Av
0,NaN,NaN,topo12,0.0000,0.0000,200.0000,0.0000,0.00000,116.6046,116.5988,NaN,100.5440,299.4784,NaN,NaN
1,M1,1.0,topo3,151.4514,151.4504,351.4588,151.4557,151.45305,93.7784,93.7650,NaN,99.1892,300.8976,NaN,NaN
2,NaN,NaN,topo12,0.0020,0.0000,200.0062,0.0000,0.00000,116.6098,116.6076,NaN,100.6144,299.4772,NaN,NaN
3,NaN,NaN,topo12,50.0000,0.0000,250.0000,0.0000,0.00000,116.6078,116.6078,NaN,100.6104,299.4772,NaN,NaN
4,M1,2.0,topo3,201.4480,151.4475,1.4454,151.4446,151.44605,93.7824,93.7706,NaN,99.1926,300.8940,NaN,NaN
5,NaN,NaN,topo12,50.0010,0.0000,250.0016,0.0000,0.00000,116.6404,116.6032,NaN,100.6164,299.4696,NaN,NaN


In [10]:
n = 2  # nombre de serie par mesure rops
angles_finales = np.zeros(int(len(angles) / n )) # nombre de serie par mesure  
for i in range(0 , len(angles) , n )  : 
    somme = 0 
    for j in range(i , i + n ) : 
        somme  = somme + angles[j]
    
    angles_finales[i//n] = somme / n 
print(angles_finales)


[151.44955]


In [17]:
file.loc[: ,"distancecg"]

0    116.6046
1     93.7784
2    116.6098
3    116.6078
4     93.7824
5    116.6404
Name: distancecg, dtype: float64

In [18]:
#pour l"extraction des distances 
lenangles_finales = len(angles_finales)
lendistances_finales = lenangles_finales + 1 
distances_finales = np.zeros(lendistances_finales)
file.loc[: , "distancemoy"]  = (file.loc[: ,"distancecg"]  + file.loc[: ,"distancecd"]) / 2 
file


,station,serie,point_visé,cg,rcg,cd,rcd,moyenne,distancecg,distancecd,distancemoy,Avcg,Avcd,zocd,Av
0,NaN,NaN,topo12,0.0000,0.0000,200.0000,0.0000,0.00000,116.6046,116.5988,116.6017,100.5440,299.4784,NaN,NaN
1,M1,1.0,topo3,151.4514,151.4504,351.4588,151.4557,151.45305,93.7784,93.7650,93.7717,99.1892,300.8976,NaN,NaN
2,NaN,NaN,topo12,0.0020,0.0000,200.0062,0.0000,0.00000,116.6098,116.6076,116.6087,100.6144,299.4772,NaN,NaN
3,NaN,NaN,topo12,50.0000,0.0000,250.0000,0.0000,0.00000,116.6078,116.6078,116.6078,100.6104,299.4772,NaN,NaN
4,M1,2.0,topo3,201.4480,151.4475,1.4454,151.4446,151.44605,93.7824,93.7706,93.7765,99.1926,300.8940,NaN,NaN
5,NaN,NaN,topo12,50.0010,0.0000,250.0016,0.0000,0.00000,116.6404,116.6032,116.6218,100.6164,299.4696,NaN,NaN


In [19]:
distance = file["distancemoy"]
p = len(file.rcg)
somme1 = 0 
somme2 = 0 
c = 0 
for i in range(1 , len(distance) , 3) : 
    somme1 = somme1 + distance[i]
    c = c + 1
for k in range(0 , len(distance)) : 
    somme2 = somme2 + distance[k]
 

In [20]:
d = np.zeros(2)
d[0] = round((somme2 - somme1) / (p - c),2)
d[1] = round(somme1 / c , 2)
d

array([116.61,  93.77])

In [21]:
#ajouter la correction 
hm = 44.95
rt  = 6400 * 10**(3)
k = 0.36
def distancecorr(d) : 
    return (1 - k / 1000) * (1 - hm/rt) * d

In [22]:
for i in range(len(d)) : 
    print(d[i])

116.61
93.77


In [23]:
dreduites = np.zeros(2)
for i in range(len(d)) : 
    dreduites[i] = distancecorr(d[i])
dreduites
distance

0    116.6017
1     93.7717
2    116.6087
3    116.6078
4     93.7765
5    116.6218
Name: distancemoy, dtype: float64

## elaboration de la station libre 

In [24]:
class point : 
    def __init__(self,nom,X ,Y , Z ): 
        self.nom = nom 
        self.X = X 
        self.Y = Y 
        self.Z = Z 
    def __repr__(self) : 
        return "le point :%s a pour coordonees  x : %f , y :  %f  z : %f" % (self.nom  , self.X , self.Y , self.Z)
A = point(file["point_visé"][0],364229.335 , 376274.384, 49.479)  
B = point(file["point_visé"][1],364417.630,376222.540,51.726)  
lambdaa =  roundd(angles[0])
lambdaa


151.4531

In [25]:
AM = dreduites[0]
BM = dreduites[1]
AB = round(np.linalg.norm(np.array([A.X-B.X , A.Y-B.Y])) , 2) 
BM

93.73558444935719

In [26]:
alpha = roundd(rad2grad(np.arcsin((BM/AB)*np.sin(grad2rad(lambdaa)))))
beta = roundd(rad2grad(np.arcsin((AM/AB)*np.sin(grad2rad(lambdaa)))))
alpha
alpha

21.5141

In [27]:
beta

27.0551

In [28]:
c = alpha + beta + lambdaa
c


200.0223

In [34]:
Gab = gisement(A.X , A.Y , B.X , B.Y )
Gba = gisement(B.X,B.Y,A.X,A.Y)
Gam = roundd(Gab + alpha)
Gbm = roundd(Gba - beta)


317.1045

## premiere determination 

In [35]:
XM1 =  A.X + AM*sing(Gam)
YM1 =  A.Y + AM*cosg(Gam)
M1 = point("m1" , XM1 , YM1 , 0)
print(round(M1.X , 2))
print(round(M1.Y , 2 ))

364325.1
376207.93


In [36]:
XM2 = B.X + BM*sing(Gbm)
YM2 = B.Y + BM*cosg(Gbm)
M2 = point("m2" ,XM2 , YM2 , 0 )
print(round(M2.X , 2))
print(round(M2.Y , 2 ))

364325.04
376207.95


In [32]:
print('XM : ' , M1.X )
print('YM : ' , M1.Y)
print("########################")
print('XM : ', M2.X )
print("YM : " , M2.Y)


XM :  364325.1042676454
YM :  376207.9298061149
########################
XM :  364325.0371055478
YM :  376207.9483692129


In [94]:
print( round((M1.X + M2.X) / 2,2)  , ' ' ,  round((M1.Y + M2.Y )/2 , 2))

364325.07   376207.94


## partie altimetrie 

In [26]:
file.loc[: , "zocd"]  = (file.loc[: ,"Avcg"]  + file.loc[: ,"Avcd"] - 400) /2


In [27]:
file.loc[: , "zocd"]

0    0.0112
1    0.0434
2    0.0458
3    0.0438
4    0.0433
5    0.0430
Name: zocd, dtype: float64

In [28]:
file.loc["Av"] = file.loc[: ,"Avcg"] - file.loc[: , "zocd"]


In [29]:
c = file.loc[: ,"Avcg"] - file.loc[: , "zocd"]
c

0     100.5328
1      99.1458
2     100.5686
3     100.5666
4      99.1493
5     100.5734
Av         NaN
dtype: float64

In [30]:
file["Av"] = c
file.drop("Av")

,station,serie,point_visé,cg,rcg,cd,rcd,moyenne,distancecg,distancecd,distancemoy,Avcg,Avcd,zocd,Av
0,NaN,NaN,topo12,0.0000,0.0000,200.0000,0.0000,0.00000,116.6046,116.5988,116.6017,100.5440,299.4784,0.0112,100.5328
1,M1,1.0,topo3,151.4514,151.4504,351.4588,151.4557,151.45305,93.7784,93.7650,93.7717,99.1892,300.8976,0.0434,99.1458
2,NaN,NaN,topo12,0.0020,0.0000,200.0062,0.0000,0.00000,116.6098,116.6076,116.6087,100.6144,299.4772,0.0458,100.5686
3,NaN,NaN,topo12,50.0000,0.0000,250.0000,0.0000,0.00000,116.6078,116.6078,116.6078,100.6104,299.4772,0.0438,100.5666
4,M1,2.0,topo3,201.4480,151.4475,1.4454,151.4446,151.44605,93.7824,93.7706,93.7765,99.1926,300.8940,0.0433,99.1493
5,NaN,NaN,topo12,50.0010,0.0000,250.0016,0.0000,0.00000,116.6404,116.6032,116.6218,100.6164,299.4696,0.0430,100.5734


In [30]:
file.to_excel(r"C:\Users\Walid\Downloads\station_libre.xlsx")


In [31]:
#determination des angles finales  on suit la meme logique dans les distances 
Averticale = file["Av"]
p = len(file.rcg)
somme3 = 0 
s = 0 
c1 = 0 
for i in range(1 , p-1 , 3) : 
    somme3 = somme3 + file.Av[i]
    c1 = c1 + 1
for k in range(0 , p-1) : 
    s = s + file.Av[k]
AV= np.zeros(2)
AV[0] = round((s - somme3) / (p - c1 - 1),4)
AV[1] = round(somme3 / c1 , 4)
AV

array([100.5604,  99.1476])

In [32]:
hauteur_prisme  = 1.62
hauteur_station =  1.37
Ha = A.Z
Hb = B.Z 
Hm1 = Ha - (hauteur_prisme - hauteur_station + d[0]*sing(100 - AV[0] )) 
Hm2 = Hb - (hauteur_prisme  - hauteur_station + d[1]*sing(100-AV[1]))
print(round((Hm1 + Hm2)/2,2))


50.24


## determination des precision 

In [2]:
import torch 
def f(lambdaa, d1, d2, x1, y1, x2, y2):
    lambdaa = grad2rad(lambdaa)
    pi = np.pi
    norm = torch.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    alpha = torch.asin((d2/norm) * torch.sin(lambdaa))
    beta = torch.asin((d1/norm) * torch.sin(lambdaa))
    xm1 = x1 + d1 * torch.sin(torch.atan((x2-x1)/(y2-y1)) + alpha) 
    ym1 = y1 + d1 * torch.cos(torch.atan((x2-x1)/(y2-y1)) + alpha) 
    xm2 = x2 + d2 * torch.sin(pi -torch.atan((x2-x1)/(y2-y1)) + beta) 
    ym2 = y2 + d2 * torch.cos(pi - torch.atan((x2-x1)/(y2-y1)) + beta) 
    
    # Add a return statement for the computed values, modify as needed
    return xm1, ym1, xm2, ym2

In [96]:
from torch.autograd.functional import jacobian
from torch import tensor
lambdaa=tensor(151.4531)
d1 = tensor(116.6017)
d2 = tensor(93.7717)
x1 = tensor(364229.335)
y1 = tensor(376274.384)
x2 = tensor(364417.630)
y2 = tensor(376222.540)

In [97]:
j = jacobian(f,(lambdaa,d1,d2,x1,y1,x2,y2))
print(j)

((tensor(-0.3844), tensor(-0.8215), tensor(0.2493), tensor(1.2058), tensor(0.2965), tensor(-0.2058), tensor(-0.2965)), (tensor(-0.5537), tensor(0.5702), tensor(0.3592), tensor(0.2965), tensor(1.4271), tensor(-0.2965), tensor(-0.4271)), (tensor(-0.1088), tensor(0.0567), tensor(-0.9878), tensor(0.0128), tensor(-0.0811), tensor(0.9872), tensor(0.0811)), (tensor(-0.6895), tensor(0.3597), tensor(0.1558), tensor(0.0811), tensor(-0.5143), tensor(-0.0811), tensor(1.5143)))


In [98]:
def pprint(A):
    if A.ndim==1:
        print(A)
    else:
        w = max([len(str(s)) for s in A]) 
        print(u'\u250c'+u'\u2500'*w+u'\u2510') 
        for AA in A:
            print(' ', end='')
            print('[', end='')
            for i,AAA in enumerate(AA[:-1]):
                w1=max([len(str(s)) for s in A[:,i]])
                print(str(AAA)+' '*(w1-len(str(AAA))+1),end='')
            w1=max([len(str(s)) for s in A[:,-1]])
            print(str(AA[-1])+' '*(w1-len(str(AA[-1]))),end='')
            print(']')
        print(u'\u2514'+u'\u2500'*w+u'\u2518')  
#CREDIT TO https://stackoverflow.com/questions/53126305/pretty-printing-numpy-ndarrays-using-unicode-characters

In [99]:
J = np.zeros((len(j),len(j[0])))
for i in range(len(j)):
    for k in range(len(j[0])):
        J[i][k] = float(j[i][k])
pprint(J)

┌──────────────────────────────────────────────────────────────────────────────────────┐
 [-0.38437867164611816 -0.8214811682701111 0.24930725991725922 1.2058007717132568   0.29647645354270935  -0.20580072700977325 -0.29647645354270935]
 [-0.5537357330322266  0.5702356100082397  0.3591519594192505  0.29647645354270935  1.427103877067566    -0.29647645354270935 -0.4271038770675659 ]
 [-0.10876264423131943 0.05673125013709068 -0.987787127494812  0.012795994058251381 -0.08112288266420364 0.9872040152549744   0.08112288266420364 ]
 [-0.6895235180854797  0.3596595525741577  0.15580953657627106 0.08112284541130066  -0.5142954587936401  -0.08112284541130066 1.5142954587936401  ]
└──────────────────────────────────────────────────────────────────────────────────────┘


In [100]:
# matrice de variance covariance des observation 
sigma = [(grad2rad(30*(10**(-4)))*np.pi/200)**2,0.0004**2,0.0004**2,0.05**2,0.05**2,0.05**2,0.05**2]
S = np.zeros((len(sigma), len(sigma)))
for i in range(len(sigma )) : 
    S[i][i] = sigma[i]
pprint(S)

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────┐
 [5.479261370662637e-13 0.0     0.0     0.0                   0.0                   0.0                   0.0                  ]
 [0.0                   1.6e-07 0.0     0.0                   0.0                   0.0                   0.0                  ]
 [0.0                   0.0     1.6e-07 0.0                   0.0                   0.0                   0.0                  ]
 [0.0                   0.0     0.0     0.0025000000000000005 0.0                   0.0                   0.0                  ]
 [0.0                   0.0     0.0     0.0                   0.0025000000000000005 0.0                   0.0                  ]
 [0.0                   0.0     0.0     0.0                   0.0                   0.0025000000000000005 0.0                  ]
 [0.0                   0.0     0.0     0.0                   0.0                   0.0                   0.00250000

In [70]:
J.T

array([[-0.38437867, -0.55373573, -0.10876264, -0.68952352],
       [-0.82148117,  0.57023561,  0.05673125,  0.35965955],
       [ 0.24930726,  0.35915196, -0.98778713,  0.15580954],
       [ 1.20580077,  0.29647645,  0.01279599,  0.08112285],
       [ 0.29647645,  1.42710388, -0.08112288, -0.51429546],
       [-0.20580073, -0.29647645,  0.98720402, -0.08112285],
       [-0.29647645, -0.42710388,  0.08112288,  1.51429546]])

In [101]:
q = J@np.eye(7)@J.T
q

array([[ 2.5568384 ,  0.80218113, -0.48690005, -0.47848728],
       [ 0.80218113,  3.15562182, -0.70149745, -0.68974868],
       [-0.48690005, -0.70149745,  1.97866851,  0.02701007],
       [-0.47848728, -0.68974868,  0.02701007,  3.19982667]])

In [102]:
a = J@S@J.T
pprint(a)

┌─────────────────────────────────────────────────┐
 [0.004180382956029506   0.0024205282394226344  -0.0005896466926696481 -0.0012173318342408026]
 [0.0024205282394226344  0.005987172097467822   -0.0010983208677018653 -0.0033314894259471074]
 [-0.0005896466926696481 -0.0010983208677018653 0.0024699000041589767  0.00021377461888119918]
 [-0.0012173318342408028 -0.003331489425947108  0.00021377461888119918 0.006426905550202514  ]
└─────────────────────────────────────────────────┘


In [103]:
for i in range(len(a)) : 
    print(a[i][i]**(1/2))
    

0.06465588106297451
0.07737681886371281
0.04969808853627045
0.08016798332378403
